In [1]:
import pandas as pd
import numpy as np
from pydataset import data
from env import host, user, password
url = f'mysql+pymysql://{user}:{password}@{host}/employees'

In [2]:
def get_db_url(host, username, password, database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'